In [34]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
driver = webdriver.Chrome(executable_path="C:\\Users\\TECHCARE\\Desktop\\crawl_project\\chromedriver-win64\\chromedriver.exe")

url = 'https://hangchinhhieu.vn/collections/laptop'
driver.get(url)

def get_product_links(page_source):
    soup = BeautifulSoup(page_source, 'html.parser')
    links = soup.select('h3.product-name a')
    product_links = [f'https://hangchinhhieu.vn{link["href"]}' for link in links] if links else None
    return product_links

all_product_links = []
# Lặp qua từng trang
for page_number in range(1,15): 
    # Truy cập trang
    driver.get(url + f'?page={page_number}')

    # Chờ để đảm bảo trang đã được tải đủ
    time.sleep(2)

    # Lấy dữ liệu từ trang hiện tại
    current_page_source = driver.page_source
    current_product_links = get_product_links(current_page_source)

    # In ra các link sản phẩm từ trang hiện tại
    for link in current_product_links:
        all_product_links.append(link)
print(all_product_links)
# Đóng trình duyệt khi hoàn thành
driver.quit()

['https://hangchinhhieu.vn/products/laptop-dynabook-satellite-pro-c40-h-pys37l-00x00u-b', 'https://hangchinhhieu.vn/products/laptop-dynabook-satellite-pro-c40-h-pys37l-01200u-b', 'https://hangchinhhieu.vn/products/laptop-asus-vivobook-14-x1404za-nk300w-i3-1215u', 'https://hangchinhhieu.vn/products/laptop-hp-340s-g7-224l1pa-i3-1005g1', 'https://hangchinhhieu.vn/products/laptop-asus-expertbook-b1-b1402cba-nk1535w', 'https://hangchinhhieu.vn/products/laptop-asus-vivobook-15-x1504za-nj582w', 'https://hangchinhhieu.vn/products/laptop-msi-modern-14-c7m-220vn-r5-7530u', 'https://hangchinhhieu.vn/products/laptop-asus-vivobook-14-x1404za-nk386w-i3-1215u', 'https://hangchinhhieu.vn/products/laptop-acer-aspire-5-a515-58p-35eu', 'https://hangchinhhieu.vn/products/laptop-hp-240-g10-8u7d8pa', 'https://hangchinhhieu.vn/products/laptop-acer-aspire-5-a514-56p-35x7-i3-1315u', 'https://hangchinhhieu.vn/products/laptop-dynabook-satellite-pro-c40-h-pys37l-01400u-b', 'https://hangchinhhieu.vn/products/lapto

In [35]:
print(len(all_product_links))

280


In [38]:
import requests
def get_product_info(all_product_links):
    driver.get(all_product_links)
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    product_id = soup.find('div', class_ = 'group-status').find_next('span', class_ ='status_name product-sku').text.strip()
    #PRODUCT_NAME
    try:
        product_name = soup.find('h1', class_ = 'title-product').text.strip()
    except:
        product_name = ''
        #PRODUCT_URL
    try:
        product_url = soup.find('link', {'rel': 'canonical'}).get('href')
    except:
        product_url = ''
    try:
        discounted_price = soup.find('span', class_ = 'price product-price').text.strip()
    except:
        discounted_price = ''
    try:
        original_price = soup.find('span', class_ = 'old-price').text.strip()
        if not original_price != '':
            original_price = discounted_price 
    except:  
        original_price = discounted_price
    try:
        discount_percent = soup.find('div', class_ = 'label_product').text.strip().replace('-','')
    except:
        discount_percent = ''
    # installment_payment
    new_response = requests.get('https://pages.hangchinhhieu.vn/thong-tin-tra-gop')
    new_soup = BeautifulSoup(new_response.text, 'html.parser')
    installment_payment = new_soup.find('div', {'id':'HEADLINE6'}).find('h3', class_ = 'ladi-headline ladi-transition').text.strip()

    max_promotion = 5
    promotion_values =  soup.find('div', class_ = 'product-promotion rounded-sm')
    # Tạo một từ điển để lưu các giá trị
    promotions = []
    if promotion_values:
    # Lấy danh sách các thông tin khuyến mãi
        for i, li in enumerate(promotion_values.find_all('li'), start=1):
            variable_name = f"Promotion__{i}"
            promotion_text = li.text.strip()
            promotions.append((variable_name, promotion_text))
    
    product_price = {
            'product_id': product_id,
            'product_name': product_name,
            'discounted_price': discounted_price,
            'original_price': original_price,
            'discount_percent': discount_percent,
            'installment_payment' : installment_payment,
            'shop_brand': 'Hàng chính hiệu'
        }
    # Thêm các biến khuyến mại vào dictionary
    for i in range(1, max_promotion + 1):
        key = f'promotion_{i}'
        # Nếu i vượt quá số lượng thực sự của khuyến mại, thiết lập giá trị là None
        if i <= len(promotions):
                value = promotions[i-1][1]
        else:
            value = ''

        product_price[key] = value

    product_price['product_url'] = product_url

    return product_price

driver = webdriver.Chrome(executable_path="C:\\Users\\TECHCARE\\Desktop\\crawl_project\\chromedriver-win64\\chromedriver.exe")
# Duyệt qua các đường link sản phẩm và lấy thông tin
data = []
for product_link in all_product_links:
    product_price = get_product_info(product_link)
    data.append(product_price)
    print(product_price)

driver.quit()

{'product_id': 'PYS37L-00X00U_B', 'product_name': "Laptop DynaBook Satellite Pro C40-H (PYS37L-00X00U_B) (i3-1005G1 | 4GB | 256GB | Intel UHD Graphics | 14' HD | DOS)", 'discounted_price': '7,490,000₫', 'original_price': '9,990,000₫', 'discount_percent': '25%', 'installment_payment': 'TRẢ GÓP 0% QUA THẺ TÍN DỤNG', 'shop_brand': 'Hàng chính hiệu', 'promotion_1': 'Túi xách DynaBook', 'promotion_2': '', 'promotion_3': '', 'promotion_4': '', 'promotion_5': '', 'product_url': 'https://hangchinhhieu.vn/products/laptop-dynabook-satellite-pro-c40-h-pys37l-00x00u-b'}
{'product_id': 'PYS37L-01200U_B', 'product_name': "Laptop DynaBook Satellite Pro C40-H PYS37L-01200U_B (i3-1005G1 | 8GB | 512GB | Intel UHD Graphics | 14' FHD | DOS)", 'discounted_price': '8,490,000₫', 'original_price': '13,490,000₫', 'discount_percent': '37%', 'installment_payment': 'TRẢ GÓP 0% QUA THẺ TÍN DỤNG', 'shop_brand': 'Hàng chính hiệu', 'promotion_1': 'Túi xách DynaBook', 'promotion_2': '', 'promotion_3': '', 'promotion_4

In [39]:
import pandas as pd
df = pd.DataFrame(data)
df

,product_id,product_name,discounted_price,original_price,discount_percent,installment_payment,shop_brand,promotion_1,promotion_2,promotion_3,promotion_4,promotion_5,product_url
0,PYS37L-00X00U_B,Laptop DynaBook Satellite Pro C40-H (PYS37L-00...,"7,490,000₫","9,990,000₫",25%,TRẢ GÓP 0% QUA THẺ TÍN DỤNG,Hàng chính hiệu,Túi xách DynaBook,,,,,https://hangchinhhieu.vn/products/laptop-dynab...
1,PYS37L-01200U_B,Laptop DynaBook Satellite Pro C40-H PYS37L-012...,"8,490,000₫","13,490,000₫",37%,TRẢ GÓP 0% QUA THẺ TÍN DỤNG,Hàng chính hiệu,Túi xách DynaBook,,,,,https://hangchinhhieu.vn/products/laptop-dynab...
2,X1404ZA-NK300W,Laptop ASUS VivoBook 14 X1404ZA-NK300W (i3-121...,"9,290,000₫","12,490,000₫",26%,TRẢ GÓP 0% QUA THẺ TÍN DỤNG,Hàng chính hiệu,Chuột có dây Rapoo N200,E-Voucher 200.000đ khi review sản phẩm,Vệ sinh miễn phí trong 2 năm (6 tháng/1 lần),,,https://hangchinhhieu.vn/products/laptop-asus-...
3,224L1PA,Laptop HP 340s G7 224L1PA (i3-1005G1 | 4GB | 5...,"9,490,000₫","11,490,000₫",17%,TRẢ GÓP 0% QUA THẺ TÍN DỤNG,Hàng chính hiệu,Chuột có dây Rapoo N200,RAM Lexar 4GB DDR4 3200MHz,,,,https://hangchinhhieu.vn/products/laptop-hp-34...
4,B1402CBA-NK1535W,Laptop ASUS ExpertBook B1 B1402CBA-NK1535W (i3...,"9,990,000₫","14,590,000₫",32%,TRẢ GÓP 0% QUA THẺ TÍN DỤNG,Hàng chính hiệu,Vệ sinh miễn phí trong 2 năm (6 tháng/1 lần),,,,,https://hangchinhhieu.vn/products/laptop-asus-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,83DE001MVN,Laptop Gaming Lenovo Legion Pro 7 16IRX9H 83DE...,"98,190,000₫","104,990,000₫",6%,TRẢ GÓP 0% QUA THẺ TÍN DỤNG,Hàng chính hiệu,Balo Legion Active Gaming,,,,,https://hangchinhhieu.vn/products/laptop-gamin...
276,PH18-72-908N,Laptop Acer Predator Helios 18 PH18-72-908N (i...,"104,990,000₫","104,990,000₫",,TRẢ GÓP 0% QUA THẺ TÍN DỤNG,Hàng chính hiệu,Balo Acer Predator SUV,Chuột Gaming ASUS ROG Harpe Ace Aim Lab Editio...,,,,https://hangchinhhieu.vn/products/laptop-acer-...
277,G834JY-N6039W,Laptop ASUS ROG Strix SCAR 18 G834JY-N6039W (i...,"104,990,000₫","124,990,000₫",16%,TRẢ GÓP 0% QUA THẺ TÍN DỤNG,Hàng chính hiệu,Balo ASUS ROG**,Chuột Gaming ROG Gladius III P514**,Customizable Armor Cap**,E-Voucher 200.000đ khi review sản phẩm,Vệ sinh miễn phí trong 2 năm (6 tháng/1 lần),https://hangchinhhieu.vn/products/laptop-asus-...
278,G634JZR-NM009W,Laptop Gaming ASUS ROG Strix SCAR 16 G634JZR-N...,"106,990,000₫","109,990,000₫",3%,TRẢ GÓP 0% QUA THẺ TÍN DỤNG,Hàng chính hiệu,Balo ASUS ROG**,Chuột Gaming ROG Gladius III P514**,Customizable Armor Cap**,"Bộ quà Asus (Túi xách ROG, Gối ROG, Sổ ROG, Kh...",E-Voucher 200.000đ khi review sản phẩm,https://hangchinhhieu.vn/products/laptop-asus-...


In [40]:
df.to_excel('hangchinhhieu_price.xlsx', index=False)